#### Conexión al Data Lake Storage

In [ ]:
# Pasar parámetros de conexión:
storage_account_name="s7databricksg2"
# Llave de token (acceso)
storage_account_access_key="XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"

In [ ]:
spark.conf.set("fs.azure.account.key."+storage_account_name+".blob.core.windows.net",storage_account_access_key)

In [ ]:
%fs ls 'wasbs://proyectog3@s7databricksg2.blob.core.windows.net'

path,name,size,modificationTime
wasbs://proyectog3@s7databricksg2.blob.core.windows.net/ReportePCBienes202209.csv,ReportePCBienes202209.csv,6807138,1684097002000
wasbs://proyectog3@s7databricksg2.blob.core.windows.net/ReportePCBienes202210.csv,ReportePCBienes202210.csv,5824350,1684097053000
wasbs://proyectog3@s7databricksg2.blob.core.windows.net/ReportePCBienes202211.csv,ReportePCBienes202211.csv,5874956,1684096999000
wasbs://proyectog3@s7databricksg2.blob.core.windows.net/ReportePCBienes202212.csv,ReportePCBienes202212.csv,4526473,1684097001000
wasbs://proyectog3@s7databricksg2.blob.core.windows.net/ReportePCBienes202301.csv,ReportePCBienes202301.csv,327480,1683945697000
wasbs://proyectog3@s7databricksg2.blob.core.windows.net/ReportePCBienes202302.csv,ReportePCBienes202302.csv,3216716,1683945700000
wasbs://proyectog3@s7databricksg2.blob.core.windows.net/ReportePCBienes202303.csv,ReportePCBienes202303.csv,6690275,1683945701000
wasbs://proyectog3@s7databricksg2.blob.core.windows.net/ReportePCBienes202304.csv,ReportePCBienes202304.csv,5162253,1683945700000


In [ ]:
from pyspark.sql.types import *

schema_empresas = StructType([StructField("FECHA_PROCESO",TimestampType(),True), \
                StructField("RUC_PROVEEDOR",StringType(),True), \
                StructField("PROVEEDOR",StringType(),True), \
                StructField("RUC_ENTIDAD",StringType(),True), \
                StructField("ENTIDAD",StringType(),True), \
                StructField("TIPO_PROCEDIMIENTO",StringType(),True), \
                StructField("ORDEN_ELECTRONICA",StringType(),True), \
                StructField("ORDEN_ELECTRONICA_GENERADA",StringType(),True), \
                StructField("ESTADO_ORDEN_ELECTRONICA", StringType(),True), \
                StructField("DOCUMENTO_ESTADO_OCAM",StringType(),True),\
                StructField("FECHA_FORMALIZACION",TimestampType(),True),
                StructField("FECHA_ULTIMO_ESTADO",TimestampType(),True),
                StructField("SUB_TOTAL",DecimalType(10,2),True),
                StructField("IGV",DecimalType(10,2),True),
                StructField("TOTAL",DecimalType(10,2),True),
                StructField("ORDEN_DIGITALIZADA",StringType(),True),
                StructField("DESCRIPCION_ESTADO",StringType(),True),
                StructField("DESCRIPCION_CESION_DERECHOS",StringType(),True),
                StructField("ACUERDO_MARCO",StringType(),True),
                  ])


input_path ="wasbs://proyectog3@s7databricksg2.blob.core.windows.net/ReportePCBienes*.csv"
df = spark.read.format("csv").option("header", True).option("sep", ";").schema(schema_empresas).load(input_path)

#### Datos de Empresa obtenidos mediante API

In [ ]:
import requests
import pandas as pd


# Seleccionamos la columna RUC_PROVEEDOR
df_ruc=df.select('RUC_PROVEEDOR').limit(5000)
#df.select('RUC_PROVEEDOR').distinct().limit(5000)

# Lo convertirmos a pandas
df_ruc_p=df_ruc.toPandas()

# Función que toma el RUC y usa una API pública para obtener información sobre la empresa 
def get_ruc(ruc):
    url = f'https://api.apis.net.pe/v1/ruc?numero={ruc}'
    response = requests.get(url)
    if response.status_code == 200:
        empresa = response.json()
        return empresa
    else:
        return None

# Lista que contendrá la data de las empresas obtenida de la API
empresas = []

# Iteración que llama a la función get_ruc con el número de RUC y se almacena la información en la lista empresas, siempre y cuando la respuesta de la API sea exitosa
for ruc in df_ruc_p['RUC_PROVEEDOR']:
    empresa = get_ruc(ruc)
    if empresa:
        empresas.append(empresa)
        
df_empresas_p = pd.DataFrame(empresas)
df_empresas = spark.createDataFrame(df_empresas_p)
df_empresas.count()

Out[11]: 606

##### Guardar en formato delta

In [ ]:
df_empresas.write.format("delta").mode("overwrite").option("mergeSchema", "true").save("wasbs://proyectog3@s7databricksg2.blob.core.windows.net/EmpresasBronze")
#df_empresas.write.format("delta").mode("overwrite").option("mergeSchema", "true").save("/user/hive/warehouse/EmpresasBronze") #comunity